# Cargando las librerias requeridas

In [ ]:
import numpy as np
import pandas as pd
from plotnine import *

In [ ]:
from sklearn import datasets
wine = datasets.load_wine() #Cargando el dataset de vino
print(wine.keys()) #Analizando las variables que tiene

#Escalando los datos
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() #Creamos un objeto de la clase StandardScaler
scaled_features = scaler.fit_transform(wine.data) #Transformamos los datos

# Fuzzy C-means

sklearn no ha implementado este modelo.

Utilizaremos la función c_means del paquete skfuzzy, el cual retorna múltiples valores. El único problema de esta función es que considera las filas como variable y las columnas como puntos.<br>
c_means(data, c, m, error, maxiter)<br><br>
Parámetros
* data => Data a clusterizar.
* c => Número de clusters.
* m => Parámetro difusor.
* error => Criterio de parada según el error.
* maxIter => Número máximo de iteraciones.

Valores de retorno
* cntr => Centro de los clusters.
* u => Matriz de pertenencia
* u0 => Matriz de pertenencia inicial.
* d => Matriz final de la distancia Euclidiana.
* jm => Log de la función objetivo.
* p => Número de iteraciones ejecutadas.
* fpc => Coeficiente de partición final. Varía entre 0 y 1 y valores cercano a 1 son mejores.

In [ ]:
#import sys
#!{sys.executable} -m pip install scikit-fuzzy
import skfuzzy as fuzzy
#Llamando a la función con los datos requeridos
(CentrosClusters, u, _, _, _, _, fpc) = fuzzy.cluster.cmeans(scaled_features.transpose(), c=4, m=2.0, error=0.005, maxiter=1000)

print("Centros de los clusters")
print(CentrosClusters)

#Atención, la matriz u está invertida
print("\nProbabilidad de pertenencia para el primer punto")
print(u[:,177]) 

#Obteniendo la membresía de cada punto
clusters = np.argmax(u, axis=0)
print("\nEtiqueta de cada punto")
print(clusters)

print("\nValor final del fuzzy partition coefficient")
print(fpc)

Centroides

In [ ]:
CentrosClusters
#pd.DataFrame(CentrosClusters.transpose(), index=wine.feature_names)

In [ ]:
u
#pd.DataFrame(u.transpose())

Seleccionando una etiqueta

In [ ]:
#Obteniendo la membresía de cada punto
clusters = np.argmax(u, axis=0)
clusters

Buscando el valor de K

In [ ]:
#Buscando el valor de K
fpcVal = []
numK=11
for k in range(2, numK):
    (CentrosClusters, u, _, _, _, _, fpc) = fuzzy.cluster.cmeans(scaled_features.transpose(), c=k, m=2, error=0.0005, maxiter=1000)
    fpcVal.append(fpc)
pd.DataFrame(fpcVal, index=np.arange(2,numK)).plot()
# tempDataFrame=pd.DataFrame(range(2, numK),columns=["numK"])
# tempDataFrame["fpcVal"]=fpcVal
# (ggplot(tempDataFrame)+aes(x="numK",y="fpcVal")+theme_bw()+geom_line()+labs(x="Número de clusters",y="fpc")
#  +scale_x_continuous(breaks=range(2,numK))
# )

In [ ]:
#Buscando el valor de K, usando within cluster distance
sse = []
numK=11
for k in range(1, numK):
    (CentrosClusters, u, _, _, _, _, fpc) = fuzzy.cluster.cmeans(scaled_features.transpose(), c=k, m=2, error=0.0005, maxiter=1000)
    clusters = np.argmax(u, axis=0)
    tempWCD=0
    for j in range(CentrosClusters.shape[0]):
        tempWCD+=np.square(scaled_features[clusters==j,]-CentrosClusters[j,:]).sum()
    sse.append(tempWCD)
pd.DataFrame(sse, index=np.arange(1,numK)).plot()


# tempDataFrame=pd.DataFrame(range(1,numK),columns=["numK"])
# tempDataFrame["sse"]=sse
# (ggplot(tempDataFrame)+aes(x="numK",y="sse")+theme_bw()+geom_line()+labs(x="Número de clusters",y="wcd")
#  +scale_x_continuous(breaks=range(1,numK)))

In [ ]:
#Obteniendo 3 clusters
(CentrosClusters, u, _, _, _, _, fpc) = fuzzy.cluster.cmeans(scaled_features.transpose(), c=3, m=1.4, error=0.0005, maxiter=1000)
clusters = np.argmax(u, axis=0)

#Creando el objeto y aplicando PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

tempData = pd.DataFrame(pca.fit_transform(scaled_features),columns=["PC1","PC2"])
tempData["labels"]=clusters

color = []
for i in range(tempData.shape[0]):
    rgb = (u*255).astype('int')
    color.append('#%02x%02x%02x' % (rgb[0,i],rgb[1,i],rgb[2,i]))
#tempData['Cluster']= color

ggplot(tempData, aes(x='PC1', y='PC2')) + geom_point(color=color)